In [68]:
import pandas as pd
import yfinance as yf

In [70]:
import pandas as pd
from backtesting import Strategy, Backtest
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.test import SMA


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [71]:
eth = yf.Ticker("ETH-USD")
eth_data = eth.history("1y")

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [75]:
bt = Backtest(eth_data, SmaCross, cash=10_000, commission=0.003)
stats = bt.run()
stats

Start                     2023-02-11 00:00...
End                       2024-02-11 00:00...
Duration                    365 days 00:00:00
Exposure Time [%]                   26.775956
Equity Final [$]                  8689.237153
Equity Peak [$]                  10001.820122
Return [%]                         -13.107628
Buy & Hold Return [%]               63.057436
Return (Ann.) [%]                  -13.074266
Volatility (Ann.) [%]               15.952386
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.800624
Avg. Drawdown [%]                  -10.292315
Max. Drawdown Duration      254 days 00:00:00
Avg. Drawdown Duration      129 days 00:00:00
# Trades                                    7
Win Rate [%]                        14.285714
Best Trade [%]                      13.310278
Worst Trade [%]                     -9.321367
Avg. Trade [%]                    

In [76]:
stats._strategy

<Strategy SmaCross>

In [77]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='3834', ...)